In [ ]:
!pip install -q --upgrade pip kaggle py7zr pandas scipy scikit-learn imbalanced-learn lightgbm hyperopt seaborn

<a id="start"></a>
# Plug and Play code for predicting Customer Churn

* [Data Preprocessing](#preprocess)
* [Model and Optimization of Hyperparameters](#model)
* [Model Combination Search](#combination)
* [Threshold Search](#threshold)

In [ ]:
from datetime import datetime
import gc
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier, plot_importance
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier

<a id="preprocess"></a>
## Data Preprocessing
[Table of Contents](#start)

Use `groupby` to convert multiple rows for each customer to a single row in the dataframe.

In [ ]:
# enter list of list of dataframe, dictionary of agg, and dictionary of renamed columns to be grouppedby
# example entry:
# groupby_list = [[transactions_df,
#                  {'payment_plan_days':'mean',
#                   'plan_list_price':'mean',
#                   'actual_amount_paid':'sum',
#                   'is_auto_renew':'mean',
#                   'transaction_date':'count',
#                   'is_cancel':'mean'},
#                  {'transaction_date':'number_of_transactions',
#                   'is_auto_renew':'auto_renew_percentage',
#                   'is_cancel':'cancel_percentage'}],
#                 [user_logs_df,
#                  {'date':'count',
#                   'num_25':'sum',
#                   'num_50':'sum',
#                   'num_75':'sum',
#                   'num_985':'sum',
#                   'num_100':'sum',
#                   'num_unq':'sum',
#                   'total_secs':'sum'},
#                  {'date':'number_of_user_logs'}]]
# enter name (string) of label used for person ID
id_label = ''

groupby_dfs = []

for i in groupby_list:
    df = i[0]
    agg = i[1]
    rename = i[2]
    df = (df.groupby(id_label, as_index=False)
          .agg(agg)
          .rename(columns=rename))
    groupby_dfs.append(df)

Merge dataframes if needed.

In [ ]:
# enter your dataframe here
data_df = 
# enter list of additional dataframes to be merged
merge_list = [] + groupby_dfs
# enter name (string) of label used for person ID
id_label = ''

for df in merge_list:
    data_df = pd.merge(data_df, df, on=id_label)

data_df.info()

In [ ]:
del groupby_list
del groupby_dfs
gc.collect()

Change format of dates to Datetime so they can be processed in an easier manner.

In [ ]:
# enter list of names (strings) of features that represent dates
date_features = []

for feature in date_features:
    # here we will assume the dates are on YYYYMMDD format, but you can change the format accoridngly
    data_df[feature] = pd.to_datetime(data_df[feature], format='%Y%m%d')

data_df.info()

It may be useful to create a categorical feature using the year of a Datetime feature.

In [ ]:
# enter list of names (strings) of Datetime features to create features with only the year of each date
year_features = []

for feature in year_features:
    # you can easily change the year to month or day if need be
    data_df[feature + '_year'] = pd.DatetimeIndex(data_df[feature]).year
    # we must delete the Datetime features for our model to work
    data_df.drop(feature, axis=1, inplace=True)

data_df.info()

One hot encode features with categorical values.

In [ ]:
# enter list of names (strings) of features to one hot encode
one_hot_features = []

for feature in one_hot_features:
    temp = pd.get_dummies(data_df[feature], prefix=feature)
    data_df = data_df.join(temp)
    data_df.drop(feature, axis=1, inplace=True)
    del temp

data_df.info()

It may be useful to create features that represent the percentage of a sum of numeric features that can be hard to understand the value of each number. An example of such features is, given data of user logs of a music streaming service, the number of songs that a user played less than 25%, between 25% and 50%, between 50% and 75%, and between 75% and 100% of the song length or the time spent by the user listening to new music and songs that they have already played before.

In [ ]:
# enter list of list of names (strings) of features to create percentage features
# example entry: [['songs_25', 'songs_50', 'songs_75', 'songs_100'], ['new_secs', 'replay_secs']]
percentage_features = []

for feature_list in percentage_features:
    sum_of_features = data_df[feature_list].sum(axis=1)
    for feature in feature_list:
        data_df[feature + '_percentage'] = data_df[feature] / sum_of_features
    del sum_of_features

data_df.info()

In [ ]:
data_df.head()

In [ ]:
del date_features
del year_features
del one_hot_features
del percentage_features
gc.collect()

Minimize memory usage using these functions.

In [ ]:
def change_datatype(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)

def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)

In [ ]:
change_datatype(data_df)
change_datatype_float(data_df)
data_df.info()

<a id="model"></a>
## Model and Optimization of Hyperparameters
[Table of Contents](#start)

Split data into train and test set.

In [ ]:
# enter name (string) of label used for churn
churn_label = ''
# enter name (string) of label used for person ID
id_label = ''

features = data_df[data_df.columns.drop([churn_label, id_label])]
labels = data_df[churn_label].astype('category')

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

Balance the train set if needed.

In [ ]:
print(y_train.value_counts(), '\n')
print(y_train.value_counts(normalize=True))

In [ ]:
if y_train.value_counts(normalize=True)[0] > 0.7:
    if y_train.value_counts()[0] > x_train.shape[0] * x_train.shape[1]:
        sampler = RandomUnderSampler()
    else:
        sampler = SMOTE()
    x_train, y_train = sampler.fit_resample(x_train, y_train)

In [ ]:
print(y_train.value_counts(), '\n')
print(y_train.value_counts(normalize=True))

Report the metrics of each model using these functions.

In [ ]:
def calc_tp_fp_tn_fn(y_actual, y_predicted):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_predicted)):
        if y_predicted[i] == y_actual[i] == 1:
            TP += 1
        if y_predicted[i] == 1 and y_actual[i] == 0:
            FP += 1
        if y_predicted[i] == y_actual[i] == 0:
            TN += 1
        if y_predicted[i] == 0 and y_actual[i] == 1:
            FN += 1

    return TP, FP, TN, FN

def calc_metrics(y_actual, y_predicted):
    TP, FP, TN, FN = calc_tp_fp_tn_fn(y_actual, y_predicted)

    if TP + FP != 0:
        precision = TP / (TP + FP)
    else:
        precision = 'N/A'

    if TP + FN != 0:
        recall = TP / (TP + FN)
    else:
        recall = 'N/A'

    if TN + FP != 0:
        specificity = TN / (TN + FP)
    else:
        specificity = 'N/A'

    accuracy = (TP + TN) / (TP + FP + TN + FN)
    if precision != 'N/A' and recall != 'N/A' and precision + recall != 0:
        f1 = 2 * precision * recall / (precision + recall)
        f2 = 5 * precision * recall / (4 * precision + recall)
    else:
        f1 = 'N/A'
        f2 = 'N/A'

    return precision, recall, specificity, accuracy, f1, f2

def print_f1_table(y_actual, y_predicted):
    precision, recall, specificity, accuracy, f1, f2 = calc_metrics(y_actual, y_predicted)
    print('Precision =', precision)
    print('Recall =', recall)
    print('Specificity =', specificity)
    print('Accuracy =', accuracy)
    print('F1-score =', f1)
    print('F2-score =', f2)

Create 3 models with optimized hyperparameters using HyperOpt: DecisionTreeClassifier, RandomForestClassifier, and LightBGM:

In [ ]:
f2_scorer = make_scorer(fbeta_score, beta=2)

DecisionTreeClassifier

In [ ]:
decision_tree_model = DecisionTreeClassifier()

def decision_tree_objective(params):
    decision_tree_model.set_params(**params)
    f2 = cross_val_score(decision_tree_model, x_train, y_train, scoring=f2_scorer).mean()
    return {'loss': -f2, 'status': STATUS_OK}

decision_tree_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.append(None, np.linspace(1, 32, num=32).astype(int)),
    'max_features': ['sqrt', 'log2', 0.6, 0.7, 0.8, 0.9, 1.0],
    'min_samples_leaf': np.linspace(1, 20, num=20).astype(int),
    'min_samples_split': np.linspace(2, 40, num=39).astype(int)
}

search_space = {
    'criterion': hp.choice('criterion', decision_tree_grid['criterion']),
    'max_depth': hp.choice('max_depth', decision_tree_grid['max_depth']),
    'max_features': hp.choice('max_features', decision_tree_grid['max_features']),
    'min_samples_leaf': hp.choice('min_samples_leaf', decision_tree_grid['min_samples_leaf']),
    'min_samples_split': hp.choice('min_samples_split', decision_tree_grid['min_samples_split'])
}

trials = Trials()
best_decision_tree = fmin(decision_tree_objective, search_space, algo=tpe.suggest, trials=trials, max_evals=100)
print('Best:', best_decision_tree)

In [ ]:
for hyperparam in best_decision_tree:
    best_decision_tree[hyperparam] = decision_tree_grid[hyperparam][best_decision_tree[hyperparam]]

print('Best:', best_decision_tree)

In [ ]:
decision_tree_model.set_params(**best_decision_tree)
decision_tree_model.fit(x_train, y_train)
decision_tree_predictions = decision_tree_model.predict(x_test)
print_f1_table(y_test.to_numpy(), decision_tree_predictions)

In [ ]:
del decision_tree_predictions
del decision_tree_grid
gc.collect()

RandomForestClassifier

In [ ]:
random_forest_model = RandomForestClassifier(n_jobs=-1)

def random_forest_objective(params):
    random_forest_model.set_params(**params)
    f2 = cross_val_score(random_forest_model, x_train, y_train, scoring=f2_scorer).mean()
    return {'loss': -f2, 'status': STATUS_OK}

random_forest_grid = {
    'criterion': ['gini', 'entropy'],
    'max_features': [0.6, 0.7, 0.8, 0.9, 1.0],
    'n_estimators': [100, 250, 500, 1000]
}

search_space = {
    'criterion': hp.choice('criterion', random_forest_grid['criterion']),
    'max_features': hp.choice('max_features', random_forest_grid['max_features']),
    'n_estimators': hp.choice('n_estimators', random_forest_grid['n_estimators'])
}

trials = Trials()
best_random_forest = fmin(random_forest_objective, search_space, algo=tpe.suggest, trials=trials, max_evals=25)
print('Best:', best_random_forest)

In [ ]:
for hyperparam in best_random_forest:
    best_random_forest[hyperparam] = random_forest_grid[hyperparam][best_random_forest[hyperparam]]

print('Best:', best_random_forest)

In [ ]:
random_forest_model.set_params(**best_random_forest)
random_forest_model.fit(x_train, y_train)
random_forest_predictions = random_forest_model.predict(x_test)
print_f1_table(y_test.to_numpy(), random_forest_predictions)

In [ ]:
del random_forest_predictions
del random_forest_grid
gc.collect()

LightGBM

In [ ]:
lightgbm_model = LGBMClassifier()

def lightgbm_objective(params):
    lightgbm_model.set_params(**params)
    f2 = cross_val_score(lightgbm_model, x_train, y_train, scoring=f2_scorer).mean()
    return {'loss': -f2, 'status': STATUS_OK}

lightgbm_grid = {
    'colsample_bytree': [0.8, 0.85, 0.9, 0.95, 1.0],
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.3],
    'max_depth': np.append(-1, np.linspace(1, 32, num=32).astype(int)),
    'n_estimators': [100, 250, 500, 1000],
    'num_leaves': [31, 50, 100, 200],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]
}

search_space = {
    'colsample_bytree': hp.choice('colsample_bytree', lightgbm_grid['colsample_bytree']),
    'learning_rate': hp.choice('learning_rate', lightgbm_grid['learning_rate']),
    'max_depth': hp.choice('max_depth', lightgbm_grid['max_depth']),
    'n_estimators': hp.choice('n_estimators', lightgbm_grid['n_estimators']),
    'num_leaves': hp.choice('num_leaves', lightgbm_grid['num_leaves']),
    'subsample': hp.choice('subsample', lightgbm_grid['subsample'])
}

trials = Trials()
best_lightgbm = fmin(lightgbm_objective, search_space, algo=tpe.suggest, trials=trials, max_evals=100)
print('Best:', best_lightgbm)

In [ ]:
for hyperparam in best_lightgbm:
    best_lightgbm[hyperparam] = lightgbm_grid[hyperparam][best_lightgbm[hyperparam]]

print('Best:', best_lightgbm)

In [ ]:
lightgbm_model.set_params(**best_lightgbm)
lightgbm_model.fit(x_train, y_train)
lightgbm_predictions = lightgbm_model.predict(x_test)
print_f1_table(y_test.to_numpy(), lightgbm_predictions)

In [ ]:
plot_importance(booster=lightgbm_model, figsize=(10, 20), importance_type='gain')
plt.show()

In [ ]:
del lightgbm_predictions
del lightgbm_grid
del search_space
del trials
gc.collect()

<a id="combination"></a>
## Model Combination Search
[Table of Contents](#start)

In [ ]:
kf = KFold()

f2_scores = [0, 0, 0, 0, 0, 0, 0]

for train_index, test_index in kf.split(x_train):
    x_train_cv = x_train.iloc[train_index]
    y_train_cv = y_train.iloc[train_index]
    x_test_cv = x_train.iloc[test_index]
    y_test_cv = y_train.iloc[test_index]

    decision_tree_model.set_params(**best_decision_tree)
    decision_tree_model.fit(x_train_cv, y_train_cv)
    decision_tree_predictions = decision_tree_model.predict(x_test_cv)
    f2_scores[0] += fbeta_score(y_test_cv.to_numpy(), decision_tree_predictions, beta=2) / 5
    decision_tree_probabilities = decision_tree_model.predict_proba(x_test_cv)

    random_forest_model.set_params(**best_random_forest)
    random_forest_model.fit(x_train_cv, y_train_cv)
    random_forest_predictions = random_forest_model.predict(x_test_cv)
    f2_scores[1] += fbeta_score(y_test_cv.to_numpy(), random_forest_predictions, beta=2) / 5
    random_forest_probabilities = random_forest_model.predict_proba(x_test_cv)

    lightgbm_model.set_params(**best_lightgbm)
    lightgbm_model.fit(x_train_cv, y_train_cv)
    lightgbm_predictions = lightgbm_model.predict(x_test_cv)
    f2_scores[2] += fbeta_score(y_test_cv.to_numpy(), lightgbm_predictions, beta=2) / 5
    lightgbm_probabilities = lightgbm_model.predict_proba(x_test_cv)

    combo_probabilities = (decision_tree_probabilities[:, 1] + random_forest_probabilities[:, 1]) / 2
    combo_predictions = np.around(combo_probabilities).astype(int)
    f2_scores[3] += fbeta_score(y_test_cv.to_numpy(), combo_predictions, beta=2) / 5

    combo_probabilities = (decision_tree_probabilities[:, 1] + lightgbm_probabilities[:, 1]) / 2
    combo_predictions = np.around(combo_probabilities).astype(int)
    f2_scores[4] += fbeta_score(y_test_cv.to_numpy(), combo_predictions, beta=2) / 5

    combo_probabilities = (random_forest_probabilities[:, 1] + lightgbm_probabilities[:, 1]) / 2
    combo_predictions = np.around(combo_probabilities).astype(int)
    f2_scores[5] += fbeta_score(y_test_cv.to_numpy(), combo_predictions, beta=2) / 5

    combo_probabilities = (decision_tree_probabilities[:, 1] +
                           random_forest_probabilities[:, 1] +
                           lightgbm_probabilities[:, 1]) / 3
    combo_predictions = np.around(combo_probabilities).astype(int)
    f2_scores[6] += fbeta_score(y_test_cv.to_numpy(), combo_predictions, beta=2) / 5

f2_scores

In [ ]:
models = [
    [[decision_tree_model, best_decision_tree]],
    [[random_forest_model, best_random_forest]],
    [[lightgbm_model, best_lightgbm]],
    [[decision_tree_model, best_decision_tree], [random_forest_model, best_random_forest]],
    [[decision_tree_model, best_decision_tree], [lightgbm_model, best_lightgbm]],
    [[random_forest_model, best_random_forest], [lightgbm_model, best_lightgbm]],
    [[decision_tree_model, best_decision_tree], [random_forest_model, best_random_forest], [lightgbm_model, best_lightgbm]],
]

best_model_index = f2_scores.index(max(f2_scores))
print('The best model is:')
if best_model_index == 0:
    print('Decision Tree')
elif best_model_index == 1:
    print('Random Forest')
elif best_model_index == 2:
    print('LightGBM')
elif best_model_index == 3:
    print('Decision Tree + Random Forest')
elif best_model_index == 4:
    print('Decision Tree + LightGBM')
elif best_model_index == 5:
    print('Random Forest + LightGBM')
else:
    print('Decision Tree + Random Forest + LightGBM')

In [ ]:
del decision_tree_predictions
del random_forest_predictions
del lightgbm_predictions
del combo_probabilities
del combo_predictions
gc.collect()

<a id="threshold"></a>
## Threshold Search
[Table of Contents](#start)

In [ ]:
f2_scores = [0, 0, 0, 0, 0, 0, 0, 0, 0]

for train_index, test_index in kf.split(x_train):
    x_train_cv = x_train.iloc[train_index]
    y_train_cv = y_train.iloc[train_index]
    x_test_cv = x_train.iloc[test_index]
    y_test_cv = y_train.iloc[test_index]

    if len(models[best_model_index]) > 1:
        combo_probabilities = []
        for i in models[best_model_index]:
            model = i[0]
            params = i[1]
            model.set_params(**params)
            model.fit(x_train_cv, y_train_cv)
            combo_probabilities.append(model.predict_proba(x_test_cv)[:, 1] / len(models[best_model_index]))
        probabilities = sum(combo_probabilities)
    else:
        model = models[best_model_index][0][0]
        params = models[best_model_index][0][1]
        model.set_params(**params)
        model.fit(x_train_cv, y_train_cv)
        probabilities = model.predict_proba(x_test_cv)[:, 1]

    for i in range(9):
        predictions = np.where(probabilities > (i + 1) / 10, 1, 0)
        f2_scores[i] += fbeta_score(y_test_cv.to_numpy(), predictions, beta=2) / 5

f2_scores

In [ ]:
best_threshold_index = f2_scores.index(max(f2_scores))
print('The best threshold is:', (best_threshold_index + 1) / 10)

In [ ]:
if len(models[best_model_index]) > 1:
    combo_probabilities = []
    for i in models[best_model_index]:
        model = i[0]
        params = i[1]
        model.set_params(**params)
        model.fit(x_train, y_train)
        combo_probabilities.append(model.predict_proba(x_test)[:, 1] / len(models[best_model_index]))
    probabilities = sum(combo_probabilities)
else:
    model = models[best_model_index][0][0]
    params = models[best_model_index][0][1]
    model.set_params(**params)
    model.fit(x_train, y_train)
    probabilities = model.predict_proba(x_test)[:, 1]

with sns.axes_style('whitegrid'):
    sns.set(rc={'figure.figsize': (20, 10)})
    ax = sns.histplot(data=probabilities, stat='percent', bins=10, binwidth=0.1)
    ax.set(xlabel='Probability of churn', ylabel='Number of customers')
    plt.show()

In [ ]:
predictions = np.where(probabilities > (best_threshold_index + 1) / 10, 1, 0)
print_f1_table(y_test.to_numpy(), predictions)

In [ ]:
combined = np.vstack((probabilities, y_test.to_numpy())).T
combined = combined[combined[:, 0].argsort()]
combined = combined[::-1]

churn_percentage = []

for p in range(1, 11):
    customers = round(0.1 * p * len(combined))
    churn_percentage.append(np.sum(combined[:customers], axis=0)[1] * 100 / len(combined[:customers]))

plot_df = pd.DataFrame({'customers': np.linspace(10, 100, 10), 'churn': churn_percentage})
plot_df

In [ ]:
with sns.axes_style('whitegrid'):
    sns.set(rc={'figure.figsize': (20, 10)})
    ax = sns.lineplot(x='customers', y='churn', data=plot_df)
    ax.set(xlabel='Percentage of customers', ylabel='Percentage of churn')
    plt.show()

In [ ]:
del kf
del f2_scores
del probabilities
del predictions
del churn_percentage
del plot_df
del ax
gc.collect()